<div class="alert alert-success"><h1>Building a Convolutional Neural Network in Python using Keras</h1></div>

Convolutional Neural Networks (CNNs) are a powerful class of deep neural networks primarily used for processing image data. CNNs have proven extremely effective in image classification, object detection, and facial recognition tasks. In this tutorial, we'll build a CNN using Keras and TensorFlow to classify images from the CIFAR-10 dataset into 10 distinct categories.

## Learning Objectives
By the end of this tutorial, you will:
+ Understand the basics of CNNs and their architecture.
+ Learn how to preprocess image data for deep learning.
+ Build a simple CNN using Keras and TensorFlow.
+ Train and evaluate the CNN model.

## Prerequisites
Before we begin, ensure you have:

+ Basic knowledge of Python programming (variables, functions, loops).
+ Understanding of fundamental machine learning concepts.
+ Python environment (version 3.x) with `tensorflow`, `keras`, and `matplotlib` installed.

<div class="alert alert-info"><b>Note:</b> For further insights into deep learning and model building with Keras and TensorFlow, consider exploring the LinkedIn Learning course <b>"Deep Learning with Python: Foundations"</b>.</div>

<div class="alert alert-success"><h2>1. Import and Preprocess the Data</h2></div>

Let's start by importing the data and verifying that the shape is as expected.

In [ ]:
from tensorflow import keras

(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

print('Training data shape:', train_images.shape)
print('Testing data shape:', test_images.shape)

The **CIFAR-10** dataset is a well-known benchmark dataset commonly used to evaluate the performance of computer vision models, particularly convolutional neural networks (CNNs). It consists of 60,000 color images, each of dimensions 32 x 32 pixels with 3 color channels (RGB), divided into:
+ 50,000 images for training.
+ 10,000 images for testing (evaluation).

Each image is assigned exactly one of ten distinct classes representing common objects - (0) Airplane, (1) Automobile, (2) Bird, (3) Cat, (4) Deer, (5) Dog, (6) Frog, (7) Horse, (8) Ship, and (9) Truck.

Let's preview the first 10 images in the training set to get a sense of what they look like. 

In [ ]:
import matplotlib.pyplot as plt

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

fig, axes = plt.subplots(1, 10, figsize = (15, 1))
for i, ax in enumerate(axes):
    ax.imshow(train_images[i])
    ax.set_title(class_names[train_labels[i][0]])
    ax.axis('off')
plt.show()

As you can see, the images are very low resolution and somewhat difficult to make out. They are 32 x 32 pixels in dimension. They weren't necessarily designed to be viewed on a high-resolution screen. However, we can tell what each image is by looking at the label.

Pixel values in images range from 0 to 255. Deep learning models (such as CNNs) perform better when input values are scaled to a smaller range, typically between 0 and 1. To accommodate this, we convert the data type of the image pixels to float32 and normalize the values to fall within 0 and 1 by dividing them by 255.

In [ ]:
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

For labels, we'll use one-hot encoding. Each integer label (0-9) is converted to a binary vector with one "hot" (1) position indicating the class.

In [ ]:
num_classes = 10
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

Now, `train_labels` and `test_labels` are matrices of shape (50000, 10) and (10000, 10), respectively.

<div class="alert alert-success"><h2>2. Define the Model Architecture</h2></div>

First, we create a Sequential model. This is a linear stack of layers that simplifies the construction of neural networks.

In [ ]:
model = keras.Sequential()

Next, we define the input shape of our data. The CIFAR-10 images have dimensions of 32 pixels by 32 pixels, with 3 color channels (RGB).

In [ ]:
model.add(keras.Input(shape = (32, 32, 3)))

We add a convolutional layer (Conv2D) with:
+ 32 filters: These filters detect various features (edges, textures, shapes) in the images.
+ Kernel size (3 x 3): Defines the size of the window moving across the image to extract features.
+ ReLU activation: Introduces non-linearity, allowing the CNN to learn more complex patterns.

In [ ]:
from tensorflow.keras import layers

model.add(layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'))

We add Batch Normalization to stabilize and speed up training by normalizing the layer inputs. This reduces internal covariate shift and allows the network to learn faster and more effectively.

In [ ]:
model.add(layers.BatchNormalization())

We apply MaxPooling to reduce the spatial dimensions (width and height) of feature maps, summarizing the presence of features in patches of the image. Here, the pooling size of (2,2) halves the dimensions of the feature maps.

In [ ]:
model.add(layers.MaxPooling2D(pool_size = (2,2)))

A Dropout layer randomly turns off neurons during training (here, with a probability of 25%). This reduces the risk of overfitting by preventing the network from overly relying on specific neurons.

In [ ]:
model.add(layers.Dropout(rate = 0.25))

We add a second convolutional block, similar to the first, but this time with 64 filters, followed again by Batch Normalization, MaxPooling, and Dropout.

In [ ]:
model.add(layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size = (2,2)))
model.add(layers.Dropout(rate = 0.25))

We introduce a third convolutional block with 128 filters, also followed by Batch Normalization, MaxPooling, and Dropout.

In [ ]:
model.add(layers.Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size = (2,2)))
model.add(layers.Dropout(rate = 0.25))

Before transitioning to the fully connected layers, we use a Flatten layer to reshape the three-dimensional feature maps into a one-dimensional vector.

In [ ]:
model.add(layers.Flatten())

We add a fully connected Dense layer with 512 neurons and ReLU activation. This layer integrates and interprets features extracted by convolutional layers to make meaningful predictions.

In [ ]:
model.add(layers.Dense(units = 512, activation = 'relu'))

Again, we apply Batch Normalization and Dropout to the fully connected layer to ensure stable training and reduce the risk of overfitting.

In [ ]:
model.add(layers.BatchNormalization())
model.add(layers.Dropout(rate = 0.5))

Finally, we add an output layer with 10 neurons, corresponding to the ten classes in CIFAR-10. The softmax activation function converts the output into probabilities for each class, indicating the likelihood of the input image belonging to each of the 10 categories.

In [ ]:
model.add(layers.Dense(units = 10, activation = 'softmax'))

Rather than adding each layer of the model one by one, we can also define the entire model at once as follows:

In [ ]:
model = keras.Sequential([
    keras.Input(shape = (32, 32, 3)),
    
    layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Dropout(rate = 0.25),
    
    layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Dropout(rate = 0.25),
    
    layers.Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Dropout(rate = 0.25),
    
    layers.Flatten(),
    layers.Dense(units = 512, activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(rate = 0.5),
    layers.Dense(units = 10, activation = 'softmax')
])

<div class="alert alert-success"><h2>3. Compile and Train the Model</h2></div>

Before training, we compile the model by specifying the optimizer, loss function, and metrics. For multiclass classification, the `'categorical_crossentropy'` loss function is appropriate, and we include `'accuracy'` as a metric to gauge the percentage of correct predictions.

To train the model, we call the `fit()` method and specify the training data, training labels, number of epochs (the number of times the model will iterate over the entire training dataset), batch size (the number of images processed before the model is updated), and validation split (the fraction of the training data to use for validation).

The `history` object returned by `model.fit()` contains the training and validation accuracy metrics for each epoch. Plotting this data helps us understand the model's performance across epochs.

The chart shows that training accuracy consistently improves and eventually plateaus near approximately 80%. This indicates the model learns effectively from the training data. The validation accuracy has fluctuations but generally follows the training accuracy trend, indicating the model generalizes reasonably well.

The `history` object returned by `model.fit()` also contains the training and validation loss metrics for each epoch. Plotting this data is also informative.

The chart shows that training loss consistently decreases, signaling that the model steadily becomes better at correctly predicting labels for the training data. Validation loss also generally decreases over epochs, but with notable fluctuations at earlier epochs. This initial instability could result from the model initially struggling with unseen data.

<div class="alert alert-success"><h2>4. Evaluate the Model</h2></div>

Finally, we can evaluate the model's accuracy against the test data.

The accuracy score provides an objective measurement of our model’s generalization ability (i.e., how well it classifies images it has never seen before).